In [ ]:
# API 키 설정 (실제 키로 변경하세요)
import os
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'
os.environ['SUPABASE_URL'] = 'your-supabase-url-here'
os.environ['SUPABASE_KEY'] = 'your-supabase-key-here'

In [ ]:
from typing import Dict, Any, List, Optional, Tuple
from openai import OpenAI
from supabase import create_client, Client
from sentence_transformers import CrossEncoder
import torch
from datetime import datetime
import time

CANDIDATE_POOL = 30
EMBED_MODEL = "text-embedding-3-small"
CE_MODEL = "BAAI/bge-reranker-v2-m3"
KW_BONUS_ALPHA = 1.2

KEYWORD_TRANSLATION = {
    "antiaging": ["안티에이징", "안티", "주름", "주름개선", "주름케어", "탄력", "탄력케어", "탄력개선", "어려보이는", "항산화", "리프팅", "노화", "안티에이지", "개선효과"],
    "anti_aging": ["안티에이징", "안티", "주름", "주름개선", "주름케어", "탄력", "탄력케어", "탄력개선", "어려보이는", "항산화", "리프팅", "노화", "개선효과"],
    "firming": ["탄력", "탄력케어", "탄력개선", "리프팅", "퍼밍", "탄탄", "elasticity", "firmness", "긴장", "피부탄력"],
    "whitening": ["화이트닝", "미백", "브라이트닝", "톤업", "brightening", "브라이트", "화사", "피부톤", "톤개선"],
    "brightening": ["브라이트닝", "화이트닝", "미백", "톤업", "화사", "피부톤", "밝은"],
    "nutrition": ["영양", "nutrition", "영양감", "영양공급", "영양크림", "nourishing"],
    "moisturizing": ["보습", "보습감", "보습력", "수분", "수분감", "수분공급", "촉촉", "촉촉한", "hydrating", "하이드레이팅", "moisturize"],
    "hydrating": ["수분", "수분감", "수분공급", "보습", "보습감", "촉촉", "촉촉한"],
    "nourishing": ["영양", "영양감", "영양공급", "영양크림"],
    "exfoliating": ["각질", "각질제거", "각질케어", "각질관리", "peeling", "필링"],
    "peeling": ["필링", "각질제거", "각질케어", "각질"],
    "soothing": ["진정", "수딩", "편안", "편안한", "calm", "calming", "진정효과"],
    "calming": ["진정", "수딩", "편안", "편안한", "calm"],
    "sensitive": ["민감", "민감성", "순한", "자극없는", "gentle", "센시티브", "민감성피부", "민감피부"],
    "acne": ["여드름", "트러블", "아크네", "피지", "모공", "trouble"],
    "pore": ["모공", "pore", "피지", "블랙헤드", "모공케어"],
    "sebum": ["피지", "유분", "sebum", "오일", "유분조절"],
    "dry": ["건조", "건성", "수분부족", "당김"],
    "oily": ["지성", "유분", "피지", "번들거림"],
    "premium": ["프리미엄", "고급", "럭셔리", "luxury", "하이엔드"],
    "luxury": ["럭셔리", "프리미엄", "고급", "하이엔드"],
    "glow": ["글로우", "광채", "윤기", "빛", "생기", "radiance", "래디언스"],
    "radiance": ["래디언스", "글로우", "광채", "윤기", "빛", "생기"],
    "dermatologist_tested": ["피부과", "테스트", "임상", "dermatologist", "피부과테스트", "테스트완료"],
    "dermatologisttested": ["피부과", "테스트", "임상", "dermatologist", "피부과테스트"],
    "lightweight": ["가벼운", "산뜻한", "light", "가볍게"],
    "light": ["가벼운", "산뜻한", "light"],
    "soft": ["부드러운", "소프트", "soft", "촉촉"],
    "gentle": ["순한", "부드러운", "gentle", "자극없는"],
    "rich": ["리치", "농축", "영양", "진한"],
    "creamy": ["크리미", "크림", "부드러운"],
    "absorption": ["흡수", "흡수력", "빠른흡수"],
    "nonsticky": ["끈적임없는", "산뜻한", "가벼운"],
    "nondrying": ["건조하지않은", "촉촉한", "보습"],
    "affordable": ["가성비", "가격저렴한", "합리적", "가격대비", "저렴"],
    "valueformoney": ["가성비", "가격대비", "가성비좋은"],
    "value_for_money": ["가성비", "가격대비", "합리적"],
}

WEATHER_KEYWORDS = {
    "spring": ["봄", "미세먼지", "황사", "꽃가루", "알러지", "알레르기", "진정", "보호", "보호막", "배리어", "클렌징", "세안", "딥클렌징", "민감", "민감성", "자극완화", "순한", "저자극", "항산화", "비타민C", "진정효과", "피부진정", "피부보호"],
    "summer": ["여름", "폭염", "자외선", "장마", "습도", "UV", "SPF", "PA", "선크림", "썬", "선케어", "쿨링", "시원한", "산뜻한", "가벼운", "땀", "피지", "모공", "유분", "지성", "수분", "젤", "에센스", "자외선차단", "유분조절", "모공케어", "피지조절", "청량", "청량감", "가벼운제형", "산뜻한발림성"],
    "fall": ["가을", "환절기", "일교차", "건조", "건조한", "보습", "수분", "진정", "밸런스", "장벽", "배리어", "회복", "리페어", "영양", "케어", "피부장벽", "세라마이드", "히알루론산", "크림", "보습감", "탄력", "촉촉한", "부드러운", "영양공급", "장벽리페어", "피부밀도", "밀도"],
    "winter": ["겨울", "한파", "건조", "극건조", "보습", "고보습", "수분", "크림", "오일", "밤", "농축", "리치", "영양", "영양크림", "리프팅", "탄력", "밀착", "피부장벽", "세라마이드", "촉촉한보습감", "부드러운발림성", "장벽리페어", "피부밀도", "밀도"]
}

WEATHER_PRIORITY_KEYWORDS = {
    "spring": ["진정", "민감", "순한", "저자극", "보호", "배리어", "피부보호", "클렌징", "딥클렌징", "세안", "알러지", "알레르기", "자극완화", "진정효과", "피부진정"],
    "summer": ["산뜻", "산뜻한", "가벼운", "끈적임없는", "논스티키", "쿨링", "시원한", "SPF", "PA", "자외선", "자외선차단", "선크림", "선케어", "UV", "피지", "피지조절", "모공", "모공케어", "유분조절", "지성", "젤", "청량", "청량감", "가벼운제형", "산뜻한발림성"],
    "fall": ["보습", "수분", "촉촉", "촉촉한", "건조", "장벽", "배리어", "피부장벽", "장벽리페어", "회복", "리페어", "진정", "밸런스", "영양", "영양공급", "세라마이드", "히알루론산"],
    "winter": ["보습", "고보습", "수분", "극건조", "건조", "크림", "리치", "농축", "밀착", "오일", "밤", "영양", "영양크림", "영양공급", "촉촉한보습감", "부드러운발림성", "세라마이드", "피부장벽", "장벽리페어"]
}

def normalize_list(v: Any) -> List[str]:
    if v is None: return []
    if isinstance(v, list): return [str(x).strip() for x in v if str(x).strip()]
    if isinstance(v, str):
        s = v.strip()
        if s.startswith("{") and s.endswith("}"): s = s[1:-1]
        if s.startswith("[") and s.endswith("]"):
            import json
            try: return json.loads(s)
            except: pass
        return [x.strip().strip('"') for x in s.split(",") if x.strip()]
    return [str(v).strip()]

def expand_keywords(keywords: List[str]) -> List[str]:
    expanded = []
    for kw in keywords:
        expanded.append(kw)
        normalized = kw.lower().replace("_", "").replace("-", "").replace(" ", "")
        if normalized in KEYWORD_TRANSLATION:
            expanded.extend(KEYWORD_TRANSLATION[normalized])
    return expanded

def get_current_season() -> str:
    month = datetime.now().month
    if month in [3, 4, 5]: return "spring"
    elif month in [6, 7, 8]: return "summer"
    elif month in [9, 10, 11]: return "fall"
    else: return "winter"

def keyword_bonus(user_keywords, product_content, product_keywords, skin_concerns=None, weather_keywords=None, current_season=None):
    kws = [k.strip() for k in (user_keywords or []) if k and str(k).strip()]
    if skin_concerns: kws.extend([k.strip() for k in skin_concerns if k and str(k).strip()])
    if weather_keywords: kws.extend([k.strip() for k in weather_keywords if k and str(k).strip()])
    if not kws: return 0.0, {"matched_keywords": [], "hit_count": 0, "total_keywords": 0, "priority_hits": 0}
    
    search_text = (product_content or "").lower()
    if product_keywords: search_text += " " + " ".join(product_keywords).lower()
    search_text_nospace = search_text.replace(" ", "")
    
    priority_kws = []
    if current_season and current_season in WEATHER_PRIORITY_KEYWORDS:
        priority_kws = [k.lower() for k in WEATHER_PRIORITY_KEYWORDS[current_season]]
    
    hit_count = 0.0
    matched_keywords = []
    priority_matched = []
    
    for kw in kws:
        kw_lower = kw.lower()
        kw_nospace = kw_lower.replace(" ", "")
        if kw_lower in search_text or kw_nospace in search_text_nospace:
            matched_keywords.append(kw)
            is_priority = any(pk in kw_lower or kw_lower in pk for pk in priority_kws)
            if is_priority:
                hit_count += 2.0
                priority_matched.append(kw)
            else:
                hit_count += 1.0
    
    max_possible_score = len(kws) * 2.0 if priority_kws else len(kws)
    score = hit_count / max(max_possible_score, 1)
    return score, {"matched_keywords": matched_keywords, "hit_count": int(hit_count), "total_keywords": len(kws), "priority_hits": len(priority_matched)}

def test_recommendation_with_diagnostics(user_id: str, intent: str = "regular"):
    start_time = time.time()
    supabase = create_client(os.environ.get("SUPABASE_URL"), os.environ.get("SUPABASE_KEY"))
    openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"\n🔧 GPU 사용: {device}")
    cross_encoder = CrossEncoder(CE_MODEL, device=device)
    
    print(f"\n{'='*80}\n📋 테스트: {intent.capitalize()} Intent\n{'='*80}")
    
    user_resp = supabase.table("customers").select("*").eq("user_id", user_id).execute()
    if not user_resp.data:
        print(f"❌ 유저 {user_id} 없음")
        return
    
    user_data = user_resp.data[0]
    user_keywords = normalize_list(user_data.get("keywords", []))
    skin_concerns = normalize_list(user_data.get("skin_concerns", []))
    expanded_keywords = expand_keywords(user_keywords)
    expanded_concerns = expand_keywords(skin_concerns)
    
    print(f"  원본 키워드: {user_keywords}")
    print(f"  🔍 확장된 키워드 ({len(expanded_keywords)}개): {expanded_keywords[:10]}...")
    print(f"  🩺 피부고민: {skin_concerns}")
    
    weather_keywords = []
    current_season = None
    if intent == "weather":
        current_season = get_current_season()
        weather_keywords = WEATHER_KEYWORDS.get(current_season, [])
        priority_kws = WEATHER_PRIORITY_KEYWORDS.get(current_season, [])
        print(f"  🌡️ Weather: {current_season.upper()}")
        print(f"  📌 우선순위 키워드 ({len(priority_kws)}개): {priority_kws[:8]}...")
        print(f"  🌤️ 전체 날씨 키워드 ({len(weather_keywords)}개)")
    
    all_keywords = expanded_keywords + expanded_concerns
    if weather_keywords: all_keywords += weather_keywords
    
    query_text = " ".join(all_keywords)
    embed_resp = openai_client.embeddings.create(input=query_text, model=EMBED_MODEL)
    query_embedding = embed_resp.data[0].embedding
    
    vector_resp = supabase.rpc("match_products", {"filter": {}, "match_count": CANDIDATE_POOL, "query_embedding": query_embedding}).execute()
    if not vector_resp.data:
        print("❌ 벡터 검색 결과 없음")
        return
    
    print(f"\n  📊 벡터 유사도 Top 5:")
    for i, p in enumerate(vector_resp.data[:5], 1):
        print(f"    {i}. PID {p['product_id']}: {p['similarity']:.6f}")
    
    product_ids = [p["product_id"] for p in vector_resp.data]
    products_resp = supabase.table("products").select("id, name, brand, price_final, discount_rate, keywords").in_("id", product_ids).execute()
    products_map = {p["id"]: p for p in products_resp.data}
    
    vector_products_resp = supabase.table("products_vector").select("product_id, content").in_("product_id", product_ids).execute()
    vector_content_map = {p["product_id"]: p["content"] for p in vector_products_resp.data}
    
    for vec_prod in vector_resp.data:
        pid = vec_prod["product_id"]
        if pid in products_map:
            vec_prod.update(products_map[pid])
            vec_prod["content"] = vector_content_map.get(pid, "")
    
    print(f"\n  🔥 Cross-Encoder 실행 중... ({len(vector_resp.data)} products)")
    ce_start = time.time()
    pairs = [[query_text, p.get("content", "")] for p in vector_resp.data]
    ce_scores = cross_encoder.predict(pairs, show_progress_bar=False)
    print(f"  ✅ Cross-Encoder 완료 ({time.time() - ce_start:.2f}초)")
    
    scored_products = []
    sim_map = {p["product_id"]: p["similarity"] for p in vector_resp.data}
    
    for i, p in enumerate(vector_resp.data):
        pid = p["product_id"]
        ce_score = float(ce_scores[i])
        product_content = p.get("content", "")
        product_keywords = normalize_list(p.get("keywords"))
        
        kwb, kw_details = keyword_bonus(
            user_keywords=expanded_keywords,
            product_content=product_content,
            product_keywords=product_keywords,
            skin_concerns=expanded_concerns,
            weather_keywords=weather_keywords if intent == "weather" else None,
            current_season=current_season if intent == "weather" else None
        )
        
        final_score = ce_score + KW_BONUS_ALPHA * kwb
        scored_products.append({
            "id": pid,
            "name": p.get("name", ""),
            "brand_name": p.get("brand", ""),
            "price": p.get("price_final", 0),
            "discount_rate": p.get("discount_rate", 0),
            "product_keywords": product_keywords,
            "ce_score": float(ce_score),
            "kw_bonus": float(kwb),
            "kw_details": kw_details,
            "final_score": float(final_score),
            "similarity": float(sim_map.get(pid, 0.0)),
        })
    
    if intent == "event":
        scored_products.sort(key=lambda x: x["final_score"], reverse=True)
        if len(scored_products) >= 5:
            top_5 = scored_products[:5]
            top_5.sort(key=lambda x: x["discount_rate"], reverse=True)
            scored_products = top_5 + scored_products[5:]
        print(f"\n  🎁 Event Intent: Top 5 중 할인율 우선")
    else:
        scored_products.sort(key=lambda x: x["final_score"], reverse=True)
    
    print(f"\n{'='*80}\n🔍 Top 5 제품 키워드 매칭 비교\n{'='*80}\n")
    
    for rank, p in enumerate(scored_products[:5], 1):
        kw_details = p["kw_details"]
        match_rate = kw_details["hit_count"] / (kw_details["total_keywords"] * 2) * 100 if intent == "weather" and kw_details["total_keywords"] > 0 else kw_details["hit_count"] / kw_details["total_keywords"] * 100 if kw_details["total_keywords"] > 0 else 0
        
        print(f"  {rank}위: PID {p['id']} - {p['name'][:40]}...")
        print(f"      브랜드: {p['brand_name']}")
        print(f"      제품 키워드: {p['product_keywords'][:5]}...")
        print(f"      ────────────────────────────────────────")
        print(f"      벡터 유사도: {p['similarity']:.6f}")
        print(f"      CE 점수: {p['ce_score']:.6f}")
        
        if intent == "weather" and kw_details.get("priority_hits", 0) > 0:
            print(f"      키워드 보너스: {p['kw_bonus']:.6f} (가중치 합산: {kw_details['hit_count']}, 우선순위 {kw_details['priority_hits']}개 ×2배)")
        else:
            print(f"      키워드 보너스: {p['kw_bonus']:.6f} ({match_rate:.0f}% 매칭, {kw_details['hit_count']}/{kw_details['total_keywords']})")
        
        print(f"      최종 점수: {p['final_score']:.6f}")
        
        if kw_details["matched_keywords"]:
            matched_str = "', '".join(kw_details["matched_keywords"][:15])
            print(f"      ✅ 매칭 키워드 ({len(kw_details['matched_keywords'])}개): '{matched_str}'...\n")
        else:
            print(f"      ❌ 매칭 키워드 없음\n")
    
    top_product = scored_products[0]
    kw_details = top_product["kw_details"]
    match_rate = kw_details["hit_count"] / (kw_details["total_keywords"] * 2) * 100 if intent == "weather" and kw_details["total_keywords"] > 0 else kw_details["hit_count"] / kw_details["total_keywords"] * 100 if kw_details["total_keywords"] > 0 else 0
    
    print(f"\n{'='*80}\n🏆 최종 추천 제품\n{'='*80}")
    print(f"  📦 {top_product['name']}")
    print(f"  🏷️  브랜드: {top_product['brand_name']}")
    print(f"  💰 가격: {top_product['price']:,}원 (할인: {top_product['discount_rate']}%)")
    print(f"\n  📊 점수:")
    print(f"     벡터 유사도: {top_product['similarity']:.6f}")
    print(f"     CE 점수: {top_product['ce_score']:.6f}")
    print(f"     키워드 보너스: {top_product['kw_bonus']:.6f}")
    print(f"     최종 점수: {top_product['final_score']:.6f}")
    print(f"     계산식: {top_product['final_score']:.4f} = {top_product['ce_score']:.4f} + ({KW_BONUS_ALPHA} × {top_product['kw_bonus']:.4f})")
    print(f"\n  🔍 키워드 매칭:")
    print(f"     전체 키워드: {kw_details['total_keywords']}개")
    print(f"     매칭된 키워드: {len(kw_details['matched_keywords'])}개")
    
    if intent == "weather" and kw_details.get("priority_hits", 0) > 0:
        print(f"     🌟 우선순위 키워드: {kw_details['priority_hits']}개 (2배 가중치)")
        print(f"     가중치 합산: {kw_details['hit_count']} (일반 {len(kw_details['matched_keywords']) - kw_details['priority_hits']} + 우선순위 {kw_details['priority_hits']}×2)")
    
    print(f"     매칭률: {match_rate:.1f}%")
    
    if kw_details["matched_keywords"]:
        print(f"\n  ✅ 매칭된 키워드 ({len(kw_details['matched_keywords'])}개):")
        for kw in kw_details["matched_keywords"][:20]:
            print(f"     - '{kw}'")
    
    print(f"\n  💡 평가:")
    if match_rate < 15:
        print(f"     ⚠️ 키워드 매칭 낮음 ({match_rate:.0f}%)")
    elif match_rate < 30:
        print(f"     ✅ 키워드 매칭 보통 ({match_rate:.0f}%)")
    else:
        print(f"     🎯 키워드 매칭 우수 ({match_rate:.0f}%)")
    
    if top_product['ce_score'] > 0.6:
        print(f"     ✅ 추천 타당함 - Cross-Encoder 점수 높음")
    else:
        print(f"     ⚠️ 재검토 권장")
    
    print(f"\n⏱️  소요시간: {time.time() - start_time:.2f}초\n")

print("\n" + "🔵" * 40)
print("시나리오 1: Regular Intent")
print("🔵" * 40)
test_recommendation_with_diagnostics("user_001", intent="regular")

print("\n\n" + "🟡" * 40)
print("시나리오 2: Weather Intent (우선순위 키워드 2배 가중치)")
print("🟡" * 40)
test_recommendation_with_diagnostics("user_001", intent="weather")

print("\n\n" + "🎁" * 40)
print("시나리오 3: Event Intent (Top 5 중 할인율 우선)")
print("🎁" * 40)
test_recommendation_with_diagnostics("user_001", intent="event")

print("\n✅ 테스트 완료!")